In [1]:
!pip install  google-generativeai sentence-transformers faiss-cpu python-pptx pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from pptx import Presentation
import fitz  # PyMuPDF
import os

# Authenticate Gemini API
GOOGLE_API_KEY = "your_gemini_api_key_here"
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

folder_path = '/content/drive/MyDrive/CTSE Lecture Notes/'
files = os.listdir(folder_path)

def extract_text_from_pptx(file_path):
    prs = Presentation(file_path)
    return "\n".join(shape.text for slide in prs.slides for shape in slide.shapes if hasattr(shape, "text"))

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    return "\n".join(page.get_text() for page in doc)

all_texts = []
for file in files:
    full_path = os.path.join(folder_path, file)
    if file.endswith(".pptx"):
        all_texts.append(extract_text_from_pptx(full_path))
    elif file.endswith(".pdf"):
        all_texts.append(extract_text_from_pdf(full_path))

full_text = "\n".join(all_texts)


Mounted at /content/drive


In [4]:
def chunk_text(text, chunk_size=300):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

text_chunks = chunk_text(full_text)

embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedder.encode(text_chunks, convert_to_numpy=True)

embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
def retrieve_relevant_chunks(question, top_k=5, token_limit=1200):
    question_emb = embedder.encode([question], convert_to_numpy=True)
    distances, indices = index.search(question_emb, top_k)

    context = ""
    total_words = 0
    for idx in indices[0]:
        chunk = text_chunks[idx]
        words = chunk.split()
        if total_words + len(words) <= token_limit:
            context += chunk + "\n"
            total_words += len(words)
        else:
            break
    return context


In [8]:
def answer_question(question):
    context = retrieve_relevant_chunks(question)

    prompt = f"""
You are a helpful academic assistant. Your job is to answer questions strictly based on lecture slide content.

Below is the content from the slides:
\"\"\"
{context}
\"\"\"

Question:
{question}

Instructions:
- If the answer to the question is clearly mentioned in the lecture content, answer using only that.
- If the topic is somewhat related but not fully covered, you may use general knowledge to support the answer.
- If the question is unrelated or the answer is not found in the content, respond with: "This information is not available in the provided lecture content."

Be accurate, concise, and avoid including any external or unrelated information.
"""

    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content([prompt])

    return response.text


In [9]:
while True:
    q = input("Ask a question about CTSE lectures (or type 'exit' to quit): ")
    if q.lower() == "exit":
        break
    print("\nAnswer:\n", answer_question(q))


Ask a question about CTSE lectures (or type 'exit' to quit): design patterns

Answer:
 The lecture content mentions the following design patterns:

*   Event Sourcing Pattern
*   External Configuration Store Pattern
*   Federated Identity Pattern
*   Health Endpoint Monitoring Pattern
*   Throttling Pattern
*   Priority Queue Pattern
*   Pipes & Filters Pattern
*   Sharding Pattern
*   Valet Key Pattern
*   Gatekeeper Pattern
*   Circuit Breaker Pattern
*   Compensating Transaction Pattern
*   Saga Pattern

Ask a question about CTSE lectures (or type 'exit' to quit): prompt engineering

Answer:
 Prompt engineering involves crafting prompts to direct what the LLM should focus on. Good prompts lead to better and more reliable outputs. It shapes LLM behavior dynamically through careful wording, like writing a mini-program through text. Techniques include system prompts, chain-of-thought prompting, few-shot prompting, zero-shot prompting, and tool-use prompting. Effective prompts should be

KeyboardInterrupt: Interrupted by user